In [2]:
import numpy as np
import pandas as pd
import re
import spacy
import unicodedata
from nltk import pos_tag, word_tokenize, FreqDist
from nltk.stem import WordNetLemmatizer
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer

## Read in debate transcript from csv

In [3]:
df = pd.read_csv('data/us_election_2020_1st_presidential_debate.csv')
df

,speaker,minute,text
0,Chris Wallace,01:20,Good evening from the Health Education Campus ...
1,Chris Wallace,02:10,This debate is being conducted under health an...
2,Vice President Joe Biden,02:49,"How you doing, man?"
3,President Donald J. Trump,02:51,How are you doing?
4,Vice President Joe Biden,02:51,I’m well.
...,...,...,...
784,Chris Wallace,01:10:43,"Gentlemen, just say that’s the end of it [cros..."
785,President Donald J. Trump,01:10:47,I want to see an honest ballot count.
786,Chris Wallace,01:10:48,We’re going to leave it there-
787,President Donald J. Trump,01:10:49,And I think he does too-


## Data Cleaning
Rename speaker column for conciseness and remove timestamp information

In [4]:
df['speaker'] = df['speaker'].replace({'President Donald J. Trump': 'Trump', 'Vice President Joe Biden': 'Biden', 'Chris Wallace': 'Wallace'})
df = df.drop('minute', 1)

In [5]:
df

,speaker,text
0,Wallace,Good evening from the Health Education Campus ...
1,Wallace,This debate is being conducted under health an...
2,Biden,"How you doing, man?"
3,Trump,How are you doing?
4,Biden,I’m well.
...,...,...
784,Wallace,"Gentlemen, just say that’s the end of it [cros..."
785,Trump,I want to see an honest ballot count.
786,Wallace,We’re going to leave it there-
787,Trump,And I think he does too-


## Text Preprocessing

Several steps need to be taken for preprocessing the transcript texts. 
If we inspect the transcripts closely, we see numeric data scattered throughout them, as well as metadata indicating where crosstalk has occurred. We will remove such information when constructing the feature vectors.

In [6]:
df[df['text'].str.contains('\[crosstalk.*\]', regex=True)]

,speaker,text
58,Wallace,[crosstalk 00:11:26] when I finish I’m going t...
114,Trump,You just lost the left. You agreed with Bernie...
133,Biden,He sends out wishful thinking. He has Executiv...
221,Trump,"Tell that to Nancy Pelosi, and Schumer [crosst..."
269,Biden,"By the way, did you see one of the last big ra..."
...,...,...
760,Trump,"You know it can’t be done. You know it can’t, ..."
761,Biden,Mail service delivers [crosstalk 01:07:21] 185...
762,Wallace,"We can keep talking. In eight states, [crossta..."
781,Wallace,I asked you. You had an opportunity to respond...


Our text preprocessing steps will involve the following:
- lowercasing text
- convert all text to closest ascii equivalent
- removing crosstalk metadata
- replace apostrophes with standard lexicons
- removing symbols and numeric data
- lemmatization

In [7]:
nlp = spacy.load("en_core_web_sm")

In [14]:
unimportant_words = ['hey', 'hello', 'hi', 'yeah', 'oh', 'number', 'like']

# Preprocess text to remove metadata and invalid characters
def preprocess(txt):
    # Replace smart quotes with standard quotes
    txt=txt.replace(u'’', u"'")

    # Convert remaining non-ascii chars to ascii
    txt = unicodedata.normalize('NFKD', txt).encode('ascii', 'ignore').decode('utf-8')
    
    # Remove crosstalk metadata
    txt = re.sub('\[crosstalk (\d\d\:)?\d\d\:\d\d\:\d\d\\]', ' ', txt)
    
    return txt

    
# Clean tokens for clustering
def cleanText(txt):
    txt = str(txt)
    
    # Lowercase text
    txt = txt.lower()
    
    # Replace apostrophes with standard lexicons
    txt = txt.replace("isn't", "is not")
    txt = txt.replace("aren't", "are not")
    txt = txt.replace("ain't", "am not")
    txt = txt.replace("won't", "will not")
    txt = txt.replace("didn't", "did not")
    txt = txt.replace("shan't", "shall not")
    txt = txt.replace("haven't", "have not")
    txt = txt.replace("hadn't", "had not")
    txt = txt.replace("hasn't", "has not")
    txt = txt.replace("don't", "do not")
    txt = txt.replace("wasn't", "was not")
    txt = txt.replace("weren't", "were not")
    txt = txt.replace("doesn't", "does not")
    txt = txt.replace("'s", " is")
    txt = txt.replace("'re", " are")
    txt = txt.replace("'m", " am")
    txt = txt.replace("'d", " would")
    txt = txt.replace("'ll", " will")
    txt = txt.replace("'ve", " have")
    
    # Lemmatization using spaCy
    doc = nlp(txt)
    txt = ' '.join([token.lemma_ for token in doc if not token.is_stop and token.lemma_ not in unimportant_words])
    
    # Remove pronouns detected by spacy
    txt = txt.replace("-PRON-", " ")
    
    # Remove symbols and numerics
    txt = re.sub(r'[^A-Za-z0-9\s]',r' ',txt)
    txt = re.sub(r'\n',r' ',txt)
    txt = re.sub(r'[0-9]',r' ',txt)
    
    # Remove punctuation from text
    txt = ''.join([c for c in txt if c not in punctuation])
    txt = ' '.join([c for c in txt.split()])
    
    return txt

df['text'] = df['text'].map(lambda x: preprocess(x))
df['cluster tokens'] = df['text'].map(lambda x: cleanText(x))

df['text'] = df['text'].str.strip()
df['cluster tokens'] = df['cluster tokens'].str.strip()
# df = df[df['cleaned'].str.len() > 0]
df.reset_index(drop=True, inplace=True)

df.to_csv('data/preprocessed.csv')
df 

,speaker,text,cluster tokens
0,Wallace,Good evening from the Health Education Campus ...,good evening health education campus case west...
1,Wallace,This debate is being conducted under health an...,debate conduct health safety protocol design c...
2,Biden,"How you doing, man?",man
3,Trump,How are you doing?,
4,Biden,I'm well.,
...,...,...,...
784,Wallace,"Gentlemen, just say that's the end of it . Th...",gentleman end end debate
785,Trump,I want to see an honest ballot count.,want honest ballot count
786,Wallace,We're going to leave it there-,go leave there
787,Trump,And I think he does too-,think too


In [15]:
biden_df = df[df['speaker'] == 'Biden']
biden_df.drop(columns=['cluster tokens'], inplace=True)
biden_df.reset_index(drop=True, inplace=True)
biden_df.to_csv('data/biden.csv')
biden_df

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,speaker,text
0,Biden,"How you doing, man?"
1,Biden,I'm well.
2,Biden,"Well, first of all, thank you for doing this a..."
3,Biden,The American people have a right to have a say...
4,Biden,"Now, what's at stake here is the President's m..."
...,...,...
244,Biden,Five states have had mail-in ballots for the l...
245,Biden,I am concerned that any court would settle thi...
246,Biden,Mail service delivers 185 million pieces of ...
247,Biden,Yes. And here's the deal. We count the ballots...


In [16]:
trump_df = df[df['speaker'] == 'Trump']
trump_df.drop(columns=['cluster tokens'], inplace=True)
trump_df.reset_index(drop=True, inplace=True)
trump_df.to_csv('data/trump.csv')
trump_df

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,speaker,text
0,Trump,How are you doing?
1,Trump,"Thank you very much, Chris. I will tell you ve..."
2,Trump,And we won the election and therefore we have ...
3,Trump,"Thank you, Joe."
4,Trump,There aren't a hundred million people with pre...
...,...,...
309,Trump,You think that's good?
310,Trump,It's already been established. Take a look at ...
311,Trump,I want to see an honest ballot cut-
312,Trump,I want to see an honest ballot count.
